In [117]:
import numpy as np
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule
import struct
from PIL import Image

# Конфигурация изображения
WIDTH = 800
HEIGHT = 600
MAX_DEPTH = 5

# Сохранение изображения в BMP
def save_bmp(filename, image, width, height):
    with open(filename, 'wb') as f:
        # Заголовок BMP
        header = bytearray([
            66, 77,   # Signature 'BM'
            0, 0, 0, 0,  # File size (will be written later)
            0, 0,     # Reserved
            0, 0,     # Reserved
            54, 0, 0, 0,  # Offset to pixel array
            40, 0, 0, 0,  # DIB header size
            *struct.pack('<i', width),   # Width
            *struct.pack('<i', height),  # Height
            1, 0,    # Color planes
            24, 0,   # Bits per pixel
            0, 0, 0, 0,  # Compression
            0, 0, 0, 0,  # Image size (0 for uncompressed)
            0, 0, 0, 0,  # X pixels per meter
            0, 0, 0, 0,  # Y pixels per meter
            0, 0, 0, 0,  # Colors in color table
            0, 0, 0, 0   # Important color count
        ])

        # Размер файла
        file_size = len(header) + len(image)
        header[2:6] = struct.pack('<I', file_size)
        f.write(header)

        # Записываем пиксели
        for row in range(height - 1, -1, -1):  # BMP записывает строки снизу вверх
            for col in range(width):
                idx = (row * width + col) * 3
                b, g, r = image[idx], image[idx + 1], image[idx + 2]
                f.write(bytes([b, g, r]))

# Определение CUDA ядра
mod = SourceModule("""
    #include <math.h>
    #define MAX_DEPTH 5
    #define PI 3.14159265358979323846

    struct Vec3 {
        float x, y, z;

        __device__ Vec3 operator+(const Vec3& v) const {
            return {x + v.x, y + v.y, z + v.z};
        }

        __device__ Vec3 operator-(const Vec3& v) const {
            return {x - v.x, y - v.y, z - v.z};
        }

        __device__ Vec3 operator*(float scalar) const {
            return {x * scalar, y * scalar, z * scalar};
        }

        __device__ Vec3 operator*(const Vec3& v) const {
            return {x * v.x, y * v.y, z * v.z};
        }

        __device__ float dot(const Vec3& v) const {
            return x * v.x + y * v.y + z * v.z;
        }

        __device__ Vec3 normalize() const {
            float len = sqrtf(dot(*this));
            return (len > 0) ? *this * (1.0f / len) : Vec3{0.0f, 0.0f, 0.0f};
        }

        __device__ Vec3& operator+=(const Vec3& v) {
            x += v.x;
            y += v.y;
            z += v.z;
            return *this;
        }
    };

    struct Sphere {
        Vec3 center;
        float radius;
        Vec3 color;
    };

    struct Light {
        Vec3 position;
        Vec3 intensity;
    };

    struct Plane {
        Vec3 point;
        Vec3 normal;
        Vec3 color;
    };

    __device__ bool intersect(const Vec3& rayOrigin, const Vec3& rayDir, const Sphere& sphere, float& t) {
        Vec3 oc = rayOrigin - sphere.center;
        float a = rayDir.dot(rayDir);
        float b = 2.0f * oc.dot(rayDir);
        float c = oc.dot(oc) - sphere.radius * sphere.radius;
        float discriminant = b * b - 4 * a * c;

        if (discriminant < 0) return false;
        t = (-b - sqrtf(discriminant)) / (2.0f * a);
        return t >= 0;
    }

    __device__ bool intersectPlane(const Vec3& rayOrigin, const Vec3& rayDir, const Plane& plane, float& t) {
        float denom = plane.normal.dot(rayDir);
        if (fabs(denom) > 1e-6f) {
            Vec3 p0l0 = plane.point - rayOrigin;
            t = p0l0.dot(plane.normal) / denom;
            return (t >= 0);
        }
        return false;
    }

    __device__ Vec3 traceRay(const Vec3& rayOrigin, const Vec3& rayDir, Sphere* spheres, int numSpheres, Light* lights, int numLights, Plane* planes, int numPlanes, int depth) {
        if (depth > MAX_DEPTH) return {0.0f, 0.0f, 0.0f};

        float closestT = 1e20f;
        int closestSphere = -1;
        int closestPlane = -1;
        bool hitPlane = false;

        for (int i = 0; i < numSpheres; ++i) {
            float t;
            if (intersect(rayOrigin, rayDir, spheres[i], t) && t < closestT) {
                closestT = t;
                closestSphere = i;
            }
        }

        for (int i = 0; i < numPlanes; ++i) {
            float t;
            if (intersectPlane(rayOrigin, rayDir, planes[i], t) && t < closestT) {
                closestT = t;
                closestPlane = i;
                hitPlane = true;
            }
        }

        Vec3 color = {0.0f, 0.0f, 0.0f};

        if (hitPlane && closestPlane != -1) {
            const Plane& plane = planes[closestPlane];
            Vec3 intersection = rayOrigin + rayDir * closestT;
            Vec3 normal = plane.normal.normalize();

            color = plane.color * 0.1f;

            for (int i = 0; i < numLights; ++i) {
                Vec3 lightDir = (lights[i].position - intersection).normalize();
                float brightness = fmaxf(0.0f, normal.dot(lightDir));
                color += plane.color * (lights[i].intensity * brightness);
            }
        }

        if (closestSphere != -1) {
            const Sphere& sphere = spheres[closestSphere];
            Vec3 intersection = rayOrigin + rayDir * closestT;
            Vec3 normal = (intersection - sphere.center).normalize();

            color = sphere.color * 0.1f;

            for (int i = 0; i < numLights; ++i) {
                Vec3 lightDir = (lights[i].position - intersection).normalize();
                bool inShadow = false;

                for (int j = 0; j < numSpheres; ++j) {
                    float shadowT;
                    if (intersect(intersection + normal * 1e-4f, lightDir, spheres[j], shadowT)) {
                        inShadow = true;
                        break;
                    }
                }
                if (!inShadow) {
                    float brightness = fmaxf(0.0f, normal.dot(lightDir));
                    color += sphere.color * (lights[i].intensity * brightness);
                }
            }
        }

        return color;
    }

__global__ void renderKernel(unsigned char* image, int width, int height, Sphere* spheres, int numSpheres, Light* lights, int numLights, Plane* planes, int numPlanes) {
    int x = blockIdx.x * blockDim.x + threadIdx.x;
    int y = blockIdx.y * blockDim.y + threadIdx.y;

    if (x >= width || y >= height) return;

    Vec3 rayOrigin = {0.0f, 0.0f, 0.0f};
    Vec3 rayDir = {(2.0f * (x + 0.5f) / width - 1.0f) * width / height, (1.0f - 2.0f * (y + 0.5f) / height), -1.0f};
    rayDir = rayDir.normalize();

    Vec3 color = traceRay(rayOrigin, rayDir, spheres, numSpheres, lights, numLights, planes, numPlanes, 0);

    int pixelIndex = (y * width + x) * 3;
    image[pixelIndex] = (unsigned char)(fminf(1.0f, color.x) * 255);
    image[pixelIndex + 1] = (unsigned char)(fminf(1.0f, color.y) * 255);
    image[pixelIndex + 2] = (unsigned char)(fminf(1.0f, color.z) * 255);
}
""")

# Настройка данных
spheres = np.array([
    (np.array([0.0, -1.0, -3.0], dtype=np.float32), 1.0, np.array([1.0, 0.0, 0.0], dtype=np.float32)),  # Сфера 1
    (np.array([2.0, 0.0, -5.0], dtype=np.float32), 1.0, np.array([0.0, 1.0, 0.0], dtype=np.float32)),  # Сфера 2
], dtype=[('center', np.float32, 3), ('radius', np.float32), ('color', np.float32, 3)])

lights = np.array([
    (np.array([2.0, 2.0, -3.0], dtype=np.float32), np.array([1.0, 1.0, 1.0], dtype=np.float32)),  # Свет 1
], dtype=[('position', np.float32, 3), ('intensity', np.float32, 3)])

planes = np.array([
    (np.array([0.0, -2.0, 0.0], dtype=np.float32), np.array([0.0, 1.0, 0.0], dtype=np.float32), np.array([0.5, 0.5, 0.5], dtype=np.float32)),  # Плоскость
], dtype=[('point', np.float32, 3), ('normal', np.float32, 3), ('color', np.float32, 3)])

# Выделение памяти на GPU
spheres_gpu = cuda.mem_alloc(spheres.nbytes)
cuda.memcpy_htod(spheres_gpu, spheres)

lights_gpu = cuda.mem_alloc(lights.nbytes)
cuda.memcpy_htod(lights_gpu, lights)

planes_gpu = cuda.mem_alloc(planes.nbytes)
cuda.memcpy_htod(planes_gpu, planes)

# Создание изображения
image = np.zeros(WIDTH * HEIGHT * 3, dtype=np.uint8)

# Настройка сетки для выполнения ядра
block_size = (16, 16, 1)
grid_size = (WIDTH // block_size[0], HEIGHT // block_size[1])

# Запуск ядра
render_kernel = mod.get_function("renderKernel")
render_kernel(
    cuda.Out(image),
    np.int32(WIDTH),
    np.int32(HEIGHT),
    spheres_gpu,
    np.int32(spheres.shape[0]),
    lights_gpu,
    np.int32(lights.shape[0]),
    planes_gpu,
    np.int32(planes.shape[0]),
    block=block_size,
    grid=grid_size
)

# Сохранение изображения
save_bmp('output.bmp', image, WIDTH, HEIGHT)


C:\Users\SONDER\AppData\Local\Temp\ipykernel_18300\338938811.py:49: UserWarning: The CUDA compiler succeeded, but said the following:
kernel.cu

  mod = SourceModule("""
